### User Interface with Gradio for QG & QA
T5 Answer Agnostic

In [1]:
import gradio as gr
from transformers import pipeline, BartForConditionalGeneration, AutoTokenizer
import Levenshtein
import os
import markdown
from bs4 import BeautifulSoup
import pickle
from utils import Chapter, File

SAVE_TREE_DIR = os.path.join("C:", os.sep, "Users", "ManuV", "Documents", "Bachelorproef", "BP_Info_Support", "data")
QG_MODEL_NAME = "ManuVleuBeu/T5_ag_SQuAD"
QA_MODEL_NAME = "damapika/roberta-base_mod_squad"

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_guidance_framework():
    with open(os.path.join(SAVE_TREE_DIR,"gf_structure.pkl"), "rb") as f:
        return pickle.load(f)
    
root = get_guidance_framework()
    
def get_all_files():
    chapters = root.get_all_chapters()

    all_files = []
    for chapter in chapters:
        all_files.extend(chapter.files)

    return all_files

files = get_all_files()

def clean_file_dirs():
    for file in files:
        file_dir = '/'.join(file.dir.split('docs\\')[1:])
        file.dir = file_dir.replace('\\','/')

    return files

files = clean_file_dirs()

def get_contexts_from_file_dir(file_dir):
    file = [file for file in files if file.dir == file_dir]
    if not len(file) == 0:
        file = file[0]
        # Sorts text per h-elem by biggest
        key_ranking = sorted(file.text.keys(), key=lambda k: len(' '.join(file.text[k])), reverse=True)
        return [' '.join(file.text[key]) for key in key_ranking]
    else:
        raise Exception("File directory does not exist!")


In [3]:


files = get_all_files()

# Question Generation Model
question_generator = pipeline("text2text-generation", model=QG_MODEL_NAME)
# question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qa-qg-hl")

# Answer Generation Model
answer_generator = pipeline("question-answering", model=QA_MODEL_NAME)
# TODO: Add all chapter contexts or an extra function where user selects chapter->context
contexts = [file.dir for file in files]
selected_context = []
generated_question = []

def calculate_similarity(answer1, answer2):
    similarity_score = 1 - (Levenshtein.distance(answer1.lower(), answer2.lower()) / max(len(answer1), len(answer2)))
    return similarity_score

def generate_question(file_dir):
    global selected_context
    global generated_question
    contexts = get_contexts_from_file_dir(file_dir)
    selected_context.clear()
    generated_question.clear()
    for context in contexts:
        question = question_generator(context)[0]['generated_text']
        question = question.split('?')[0]+'?'# Selects first question
        # questions = [question.strip() for question in string.split('?') if question.endswith('?')] instead of first question takes multiple per context
        generated_question.append(question)
    selected_context = contexts
    return '\n'.join(generated_question)

# Interface for QG
io1 = gr.Interface(
    fn=generate_question,
    inputs=[
        gr.inputs.Dropdown(choices=contexts, label="Select a context", type="value"),
    ],
    live=False,
    outputs=[
        gr.outputs.Textbox(label="Generated Question"),
    ],
    title="Quiz Generator",
    description="Select a context, generate a question, and compare your answer.",
    allow_flagging="never",
)

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [4]:
io1.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [6]:
def generate_answer(question,user_answer):
    context = selected_context[generated_question.index(question)]
    answer = answer_generator(context=context,question=question)
    score=calculate_similarity(user_answer,answer['answer'])
    return [user_answer,answer['answer'],score]

# Additional interface for answer checking
io2 = gr.Interface(
    fn=generate_answer,
    live=False,
    inputs=[gr.inputs.Dropdown(choices=generated_question,label="Select a question",type="value"), gr.outputs.Textbox(label="User Answer")],
    outputs=[gr.outputs.Textbox(label="User Answer"),gr.outputs.Textbox(label="Generated Answer"),gr.outputs.Textbox(label="Levenshtein Answer score")],
    title="Answer Checker",
    description="Compare your answer.",
    allow_flagging="never",
    interface_id = "io2"
)

io2.launch()

c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\ManuV\AppData\Local\miniconda3\envs\quiz_generator\lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\ManuV\AppData\Local\Temp\ipykernel_16868\2082326991.py:8: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'interface_id': 'io2'}
  io2 = gr.Interface(


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
